# Optional Lab: AgentCore Policy - Secure Your Agent with Cedar Policies

## Overview

Welcome to the Amazon Bedrock AgentCore Policy hands-on lab! This notebook will guide you through implementing policy-based deterministic controls for AI agent-tool interactions.

### What is AgentCore Policy?

Amazon Bedrock AgentCore Policy enables developers to define and enforce security controls for AI agent interactions with tools by creating a protective boundary around agent operations. AI agents can dynamically adapt to solve complex problems, but this flexibility introduces security challenges:

- **Data Leakage**: Agents may inadvertently expose private information
- **Business Rule Violations**: Agents might misinterpret or bypass business rules
- **Authority Overreach**: Agents could act outside their intended scope

Policy intercepts inbound agent traffic through AgentCore Gateways and evaluates each request against defined policies before allowing tool access.

### Key Benefits

- **Declarative Security**: Define policies using Cedar language, not code
- **Runtime Enforcement**: Policies are evaluated in real-time
- **Fine-Grained Control**: From coarse restrictions to detailed access control
- **Separation of Concerns**: Security logic lives outside agent code
- **Enterprise Scale**: Deploy autonomous agents safely in production

## Prerequisites

- **Completed Lab 3**: AgentCore Gateway must be set up and working
- AWS CLI configured with appropriate credentials
- Python 3.10+ with boto3 installed

## Demo Scenario

We will implement policy controls for the customer support tools from Lab 3:

- **Tools**: web_search, check_warranty_status
- **Policy Rules**: Allow web searches and warranty checks through the gateway

## Step 0: Environment Setup

In [ ]:
import sys
from pathlib import Path
import boto3
import json
import logging
import requests
import time

from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

lab_helpers_dir = Path.cwd() / "lab_helpers"
if str(lab_helpers_dir) not in sys.path:
    sys.path.insert(0, str(lab_helpers_dir))

from lab_helpers.utils import get_ssm_parameter, put_ssm_parameter, get_cognito_client_secret, delete_ssm_parameter

REGION = boto3.session.Session().region_name or "us-east-1"

try:
    sts = boto3.client("sts")
    identity = sts.get_caller_identity()
    ACCOUNT_ID = identity['Account']
    print(f"AWS Credentials Verified - Account: {ACCOUNT_ID}, Region: {REGION}")
except Exception as e:
    print(f"AWS Credentials Error: {e}")

## Step 1: Load Existing Gateway Configuration from Lab 3

In [ ]:
print("Loading Gateway configuration from Lab 3...")

gateway_client = boto3.client("bedrock-agentcore-control", region_name=REGION)

try:
    GATEWAY_ID = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=GATEWAY_ID)
    
    GATEWAY_ARN = gateway_response["gatewayArn"]
    GATEWAY_URL = gateway_response["gatewayUrl"]
    GATEWAY_NAME = gateway_response["name"]
    GATEWAY_ROLE_ARN = get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role")
    
    print(f"Gateway Loaded: {GATEWAY_NAME}")
    print(f"Gateway ID: {GATEWAY_ID}")
    print(f"Gateway ARN: {GATEWAY_ARN}")
    print(f"Gateway Role ARN: {GATEWAY_ROLE_ARN}")
except Exception as e:
    print(f"Error: {e}")
    print("Please complete Lab 3 first.")

## Step 2: Helper Functions and Baseline Test

In [ ]:
def get_access_token():
    client_id = get_ssm_parameter("/app/customersupport/agentcore/machine_client_id")
    client_secret = get_cognito_client_secret()
    token_url = get_ssm_parameter("/app/customersupport/agentcore/cognito_token_url")
    scope = get_ssm_parameter("/app/customersupport/agentcore/cognito_auth_scope")
    
    response = requests.post(
        token_url,
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data={"grant_type": "client_credentials", "client_id": client_id, "client_secret": client_secret, "scope": scope}
    )
    response.raise_for_status()
    return response.json()["access_token"]

def list_gateway_tools(gateway_url, access_token):
    mcp_client = MCPClient(lambda: streamablehttp_client(gateway_url, headers={"Authorization": f"Bearer {access_token}"}))
    with mcp_client:
        tools = mcp_client.list_tools_sync()
        return [(tool.tool_name, getattr(tool, 'description', '')) for tool in tools]

access_token = get_access_token()
print("Access token obtained")

print("\nAvailable tools BEFORE policy:")
tools_before = list_gateway_tools(GATEWAY_URL, access_token)
for tool_name, desc in tools_before:
    print(f"  - {tool_name}")
print(f"Agent has access to {len(tools_before)} tool(s) without restrictions")

## Step 3: Create Policy Engine

A Policy Engine evaluates requests against Cedar policies in real-time.

- **LOG_ONLY**: Evaluates but does not block (for testing)
- **ENFORCE**: Actively blocks non-compliant requests

When attached in ENFORCE mode, the default action is DENY.

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient

policy_client = PolicyClient(region_name=REGION)
policy_client.logger.setLevel(logging.INFO)

print("Creating Policy Engine...")
engine = policy_client.create_or_get_policy_engine(
    name="CustomerSupportPolicyEngine",
    description="Policy engine for customer support agent tool governance",
)

POLICY_ENGINE_ID = engine['policyEngineId']
POLICY_ENGINE_ARN = engine['policyEngineArn']

print(f"Policy Engine Created: {POLICY_ENGINE_ID}")
print(f"Policy Engine ARN: {POLICY_ENGINE_ARN}")

put_ssm_parameter("/app/customersupport/agentcore/policy_engine_id", POLICY_ENGINE_ID)
put_ssm_parameter("/app/customersupport/agentcore/policy_engine_arn", POLICY_ENGINE_ARN)

## Step 3.5: Add Policy Engine Permissions to Gateway Role

Before attaching the Policy Engine to the Gateway, we need to ensure the Gateway's IAM role has the necessary permissions to access the Policy Engine. This step adds an inline policy with `bedrock-agentcore:GetPolicyEngine` and related permissions.

In [ ]:
iam_client = boto3.client("iam", region_name=REGION)

# Extract role name from ARN
gateway_role_name = GATEWAY_ROLE_ARN.split("/")[-1]

# Define the policy document for Policy Engine access
policy_engine_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PolicyEngineAccess",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetPolicyEngine",
                "bedrock-agentcore:GetPolicy",
                "bedrock-agentcore:ListPolicies",
                "bedrock-agentcore:EvaluatePolicy"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{REGION}:{ACCOUNT_ID}:policy-engine/*",
                f"arn:aws:bedrock-agentcore:{REGION}:{ACCOUNT_ID}:policy/*"
            ]
        }
    ]
}

print(f"Adding Policy Engine permissions to Gateway role: {gateway_role_name}")

try:
    iam_client.put_role_policy(
        RoleName=gateway_role_name,
        PolicyName="PolicyEngineAccessPolicy",
        PolicyDocument=json.dumps(policy_engine_policy)
    )
    print("✅ Policy Engine permissions added to Gateway role")
except Exception as e:
    print(f"⚠️ Error adding permissions: {e}")

# Wait for IAM permissions to propagate
print("Waiting 15 seconds for IAM permissions to propagate...")
time.sleep(15)
print("Done waiting.")

## Step 4: Attach Policy Engine to Gateway

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient

gateway_mgmt_client = GatewayClient(region_name=REGION)
gateway_mgmt_client.logger.setLevel(logging.INFO)

print("Attaching Policy Engine to Gateway...")
gateway_mgmt_client.update_gateway_policy_engine(
    gateway_identifier=GATEWAY_ID,
    policy_engine_arn=POLICY_ENGINE_ARN,
    mode="ENFORCE",
)
print("✅ Policy Engine attached in ENFORCE mode")
print("IMPORTANT: With empty policy engine, ALL tool access is now DENIED by default!")

## Step 5: Verify Default DENY Behavior

In [ ]:
access_token = get_access_token()

print("Available tools AFTER policy engine attachment (empty policies):")
try:
    tools_after = list_gateway_tools(GATEWAY_URL, access_token)
    if tools_after:
        for tool_name, desc in tools_after:
            print(f"  - {tool_name}")
    else:
        print("  (No tools available - all blocked by default DENY)")
except Exception as e:
    print(f"  (No tools available - blocked by default DENY)")

print("\nDefault DENY behavior confirmed")

## Step 5.5: Clean Up Existing Policies (if any)

If you've run this lab before, there may be existing policies that need to be deleted before creating new ones.

In [ ]:
# Clean up any existing policies from previous runs
print("Checking for existing policies...")

try:
    # Use boto3 directly to list and delete policies
    agentcore_client = boto3.client("bedrock-agentcore-control", region_name=REGION)
    
    policies_response = agentcore_client.list_policies(policyEngineId=POLICY_ENGINE_ID)
    existing_policies = policies_response.get('policies', [])
    
    if existing_policies:
        print(f"Found {len(existing_policies)} existing policies. Deleting...")
        for p in existing_policies:
            policy_id = p.get('policyId')
            policy_name = p.get('name', 'unknown')
            print(f"  Deleting policy: {policy_name} ({policy_id})")
            try:
                agentcore_client.delete_policy(
                    policyEngineId=POLICY_ENGINE_ID,
                    policyId=policy_id
                )
                print(f"  ✅ Deleted: {policy_name}")
            except Exception as e:
                print(f"  ⚠️ Could not delete {policy_name}: {e}")
        # Wait for deletions to propagate
        print("Waiting 5 seconds for deletions to propagate...")
        time.sleep(5)
    else:
        print("No existing policies found.")
except Exception as e:
    print(f"Note: Could not check for existing policies: {e}")

print("Ready to create new policies.")

## Step 6: Create Cedar Policies

Cedar policy structure:
```cedar
permit(principal, action, resource);
```

Our policies:
1. Web Search: Allow access to the web_search tool
2. Warranty Check: Allow access to the check_warranty_status tool

In [ ]:
print("Creating Web Search Policy...")

# Simple policy without conditions - just allow access to the tool
web_search_statement = f'permit(principal, action == AgentCore::Action::"LambdaUsingSDK___web_search", resource == AgentCore::Gateway::"{GATEWAY_ARN}");'

print(f"Policy: {web_search_statement}")

try:
    web_search_policy = policy_client.create_or_get_policy(
        policy_engine_id=POLICY_ENGINE_ID,
        name="web_search_allowed",
        description="Allow web search tool access",
        definition={"cedar": {"statement": web_search_statement}},
    )
    print(f"✅ Web Search Policy Created: {web_search_policy['policyId']}")
except Exception as e:
    print(f"⚠️ Error creating web search policy: {e}")
    print("This may happen if the policy already exists. Try running Step 5.5 first.")

In [ ]:
print("Creating Warranty Check Policy...")

warranty_statement = f'permit(principal, action == AgentCore::Action::"LambdaUsingSDK___check_warranty_status", resource == AgentCore::Gateway::"{GATEWAY_ARN}");'

print(f"Policy: {warranty_statement}")

try:
    warranty_policy = policy_client.create_or_get_policy(
        policy_engine_id=POLICY_ENGINE_ID,
        name="warranty_check_allowed",
        description="Allow warranty status checks",
        definition={"cedar": {"statement": warranty_statement}},
    )
    print(f"✅ Warranty Check Policy Created: {warranty_policy['policyId']}")
except Exception as e:
    print(f"⚠️ Error creating warranty policy: {e}")
    print("This may happen if the policy already exists. Try running Step 5.5 first.")

## Step 7: Verify Tools Are Now Accessible

In [ ]:
access_token = get_access_token()

print("Available tools AFTER policies are created:")
tools_with_policy = list_gateway_tools(GATEWAY_URL, access_token)
for tool_name, desc in tools_with_policy:
    print(f"  - {tool_name}")
print(f"\nAgent now has access to {len(tools_with_policy)} tool(s) with policy controls")

## Step 8: Test Policy Enforcement

Test scenarios:
1. Web search - should be ALLOWED
2. Warranty check - should be ALLOWED

In [ ]:
print("Creating agent with policy-controlled tools...")

mcp_client = MCPClient(lambda: streamablehttp_client(GATEWAY_URL, headers={"Authorization": f"Bearer {access_token}"}))

model = BedrockModel(model_id="us.anthropic.claude-sonnet-4-20250514-v1:0", temperature=0.3, region_name=REGION)

mcp_client.start()
tools = mcp_client.list_tools_sync()

agent = Agent(
    model=model,
    tools=tools,
    system_prompt="You are a helpful customer support assistant. Use the available tools to help customers. If a tool call fails due to policy restrictions, explain this to the user."
)
print("Agent ready!")

In [ ]:
print("TEST 1: Web Search (should be ALLOWED)")
print("="*60)
try:
    response = agent("Use the web_search tool to search for 'AWS Bedrock'")
    print("Result: ALLOWED")
except Exception as e:
    print(f"Result: {e}")

In [ ]:
print("TEST 2: Warranty Check (should be ALLOWED)")
print("="*60)
try:
    response = agent("Check the warranty status for serial number ABC123")
    print("Result: ALLOWED")
except Exception as e:
    print(f"Result: {e}")

In [ ]:
mcp_client.stop()
print("MCP client stopped")

## Congratulations!

You have successfully completed the AgentCore Policy lab!

### What You Accomplished

- Created a Policy Engine for your AgentCore Gateway
- Wrote Cedar policies to control tool access
- Tested ALLOW scenarios with real agent requests
- Understood how policies provide fine-grained access control

### Key Takeaways

1. **Default DENY**: When a Policy Engine is attached, all access is denied by default
2. **Cedar Policies**: Use declarative Cedar language to define access rules
3. **Runtime Enforcement**: Policies are evaluated in real-time for each request
4. **Separation of Concerns**: Security logic lives outside your agent code

### Cleanup

To clean up the Policy resources created in this lab, run **[Lab 7 - Cleanup](lab-07-cleanup.ipynb)**. The cleanup notebook will automatically handle Policy Engine cleanup along with all other resources.

### Additional Resources

- [Cedar Policy Language Documentation](https://docs.cedarpolicy.com/)
- [Amazon Bedrock AgentCore Policy Documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/policy.html)